# preface

# for plotting

In [ ]:
from IPython import display

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D

import time

from pyquil.quil import Program

In [ ]:
def get_fig():
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_aspect('equal')

    ax.quiver(0, 0, 0, 1, 0, 0, arrow_length_ratio=.1, color='k')
    ax.quiver(0, 0, 0, -1, 0, 0, arrow_length_ratio=.1, color='k')
    ax.quiver(0, 0, 0, 0, 1, 0, arrow_length_ratio=.1, color='k')
    ax.quiver(0, 0, 0, 0, -1, 0, arrow_length_ratio=.1, color='k')
    ax.quiver(0, 0, 0, 0, 0, 1, arrow_length_ratio=.1, color='k')
    ax.quiver(0, 0, 0, 0, 0, -1, arrow_length_ratio=.1, color='k')
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])

    ax.set_ylabel(r'$y$')
    ax.set_xlabel(r'$x$')
    ax.set_zlabel(r'$z$')
    
    ax.xaxis.pane.set_edgecolor('black')
    ax.yaxis.pane.set_edgecolor('black')
    ax.xaxis.pane.fill = False;
    ax.yaxis.pane.fill = False;
    ax.zaxis.pane.fill = False;
    ax.view_init(5, 30);
    ax.grid(False);
    
    return fig, ax

In [ ]:
cpal = sns.color_palette('viridis', 200)

# experiment

### import pennylane and the pennylane momentum optimizer

In [ ]:
import pennylane as qml
from pennylane import numpy as np

In [ ]:
from pennylane.optimize import MomentumOptimizer

### create forest QPU device, using the Aspen-1-15Q-A lattice

In [ ]:
# note that, for this example, we can book any QPU lattice, not necessarily just Aspen-1-15Q-A

dev = qml.device('forest.qpu', 
                 device='Aspen-1-15Q-A', 
                 shots=100,
                 active_reset=True)

### define the circuit and objective function

In [ ]:
@qml.qnode(dev) 
def circuit(var):
    qml.RX(var[0], wires=0) 
    qml.RY(var[1], wires=0)
    return qml.expval.PauliZ(0)

In [ ]:
def objective(var):
    return circuit(var)

### initialize optimizer object, define initial parameters and number of optimization iterations

In [ ]:
optimizer = MomentumOptimizer()

In [ ]:
rx_0 = -0.02
ry_0 = -0.02

n_iterations = 200

### for n_iterations iterations, use the momentum optimizer to find the parameters that will flip the qubit

In [ ]:
fig, ax = get_fig()

var = np.array([rx_0, ry_0])


for iteration in range(n_iterations):
    var = optimizer.step(objective, var)
    
    if iteration % 2 == 0: 
        pauli_z = objective(var)
        
        x = np.sin(var[0])*np.cos(var[1])
        y = np.sin(var[1])*np.cos(var[0])
        z = np.cos(var[0])
        qubit = ax.quiver(0, 0, 0, x, y, z, arrow_length_ratio=.1, 
                          color=cpal[iteration])

        program = Program(dev.program)
        txt = ax.text(1, 1, .5, program.out());

        display.clear_output(wait=True)
        display.display(fig)
        time.sleep(0.01)
        
        txt.remove()
        
plt.close('all')